In [1]:
import azureml
import json
from azureml.core.model import Model
from azureml.core import Workspace, Run

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  0.1.80


In [2]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#   ws.write_config()
#   print('Workspace configuration succeeded. You are all set!')
#except:
#   print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

Found the config file in: C:\projects\PyTorch-YOLOv3\aml_config\config.json


# Managing Models
This section is devoted to managin the YOLOv3 model in Azure Machine Learning service

In [3]:
models = Model.list(workspace=ws)
for m in models:
    print(m.name, m.version)

YOLOv3 5
sklearn_mnist 13
best_model 2
sklearn_mnist 12
sklearn_mnist 11
sklearn_mnist 10
sklearn_mnist 9
sklearn_mnist 8
sklearn_mnist 7
sklearn_mnist 6
sklearn_mnist 5
inception 1
sklearn_regression_model.pkl 6
sklearn_regression_model.pkl 5
YOLOv3 4
sklearn_regression_model.pkl 3
sklearn_mnist 4
sklearn_regression_model.pkl 2
YOLOv3 3
sklearn_regression_model.pkl 1
best_ridge_model 1
sklearn_mnist 3
best_model 1
AutoMLef440bcfebest 1
YOLOv3 2
sklearn_mnist 2
sklearn_mnist 1
YOLOv3 1


## Register folder as YOLOv3 model

In [4]:
model_file = './model'
m = Model.register(ws, model_name='YOLOv3', model_path=model_file)
m

Registering model YOLOv3


In [6]:
print(m.name, m.id, m.version, sep='\n')

YOLOv3
YOLOv3:6
6


# Managing Images
Creating environment file

In [8]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
with open('./requirements.txt', 'r') as r:
    deps = r.read().split('\n')
    deps = [x.rstrip().lstrip() for x in deps]
    for item in deps:
        if len(item) > 0:
            print('Adding "pip install {}"'.format(item))
            myenv.add_pip_package(item)

yml = 'yolov3.yml'
with open(yml,'w') as f:
    print('Writing out {}'.format(yml))
    f.write(myenv.serialize_to_string())
    print('Done!')

Adding "pip install scikit-image"
Adding "pip install numpy"
Adding "pip install torch>=0.4.0"
Adding "pip install torchvision"
Adding "pip install pillow"
Adding "pip install matplotlib"
Writing out yolov3.yml
Done!


Creating Image

In [9]:
from azureml.core.image import ContainerImage, Image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                    runtime="python", 
                                    conda_file="yolov3.yml",
                                    dependencies=['./models.py', './utils'], )


image = Image.create(ws, 'yolov3', [m], image_config)

Creating image


In [11]:
imgs =  ws.images['yolov3']
print(imgs.image_build_log_uri)

None


# Deploying Image
Deploy created image to ACI

In [12]:
imgs =  ws.images['yolov3']

In [13]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "COCO",  "method" : "YOLOv3"}, 
                                               description='object detection')



In [10]:
service = Webservice.deploy_from_image(workspace=ws, 
                                       image=imgs, 
                                       name='yolov3-svc', 
                                       deployment_config=aciconfig)

Creating service


In [18]:
print(service.scoring_uri)
service.wait_for_deployment(show_output=True)

None
Running....................
SucceededACI service creation operation finished, operation "Succeeded"


In [14]:
svc = ws.webservices['yolov3-svc']
svc.scoring_uri

'http://13.68.203.69:80/score'

In [15]:
logs = svc.get_logs()
print(logs)


2018-11-21 07:55:57,095 CRIT Supervisor running as root (no user in config file)
2018-11-21 07:55:57,097 INFO supervisord started with pid 1
2018-11-21 07:55:58,099 INFO spawned: 'rsyslog' with pid 10
2018-11-21 07:55:58,101 INFO spawned: 'program_exit' with pid 11
2018-11-21 07:55:58,103 INFO spawned: 'nginx' with pid 12
2018-11-21 07:55:58,201 INFO spawned: 'iot' with pid 13
2018-11-21 07:55:58,202 INFO spawned: 'gunicorn' with pid 14
2018-11-21 07:55:58,245 INFO success: iot entered RUNNING state, process has stayed up for > than 0 seconds (startsecs)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2018-11-21 07:55:58,765 INFO exited: iot (exit status 1; expected)
2018-11-21 07:55:59,582 INFO success: rsyslog entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-11-21 07:55:59,582 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-11-21 07:56:03,586 INFO success: nginx 